In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt
import os
import io
import geopandas as gpd
import seaborn as sns
# suppress warning
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [62]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
# display all columns
# avoid scientific digit
pd.options.display.float_format = '{:.8f}'.format

In [100]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
link = '/content/drive/My Drive/X999/MTA_Subway_Stations_20250502.csv'
metro_map = pd.read_csv(link)

In [5]:
metro_map.columns

Index(['GTFS Stop ID', 'Station ID', 'Complex ID', 'Division', 'Line',
       'Stop Name', 'Borough', 'CBD', 'Daytime Routes', 'Structure',
       'GTFS Latitude', 'GTFS Longitude', 'North Direction Label',
       'South Direction Label', 'ADA', 'ADA Northbound', 'ADA Southbound',
       'ADA Notes', 'Georeference'],
      dtype='object')

In [6]:
metro_map.head()

,GTFS Stop ID,Station ID,Complex ID,Division,Line,Stop Name,Borough,CBD,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Northbound,ADA Southbound,ADA Notes,Georeference
0,R01,1,1,BMT,Astoria,Astoria-Ditmars Blvd,Q,False,N W,Elevated,40.775036,-73.912034,Last Stop,Manhattan,0,0,0,NaN,POINT (-73.912034 40.775036)
1,R03,2,2,BMT,Astoria,Astoria Blvd,Q,False,N W,Elevated,40.770258,-73.917843,Astoria,Manhattan,1,1,1,NaN,POINT (-73.917843 40.770258)
2,R04,3,3,BMT,Astoria,30 Av,Q,False,N W,Elevated,40.766779,-73.921479,Astoria,Manhattan,0,0,0,NaN,POINT (-73.921479 40.766779)
3,R05,4,4,BMT,Astoria,Broadway,Q,False,N W,Elevated,40.761820,-73.925508,Astoria,Manhattan,0,0,0,NaN,POINT (-73.925508 40.76182)
4,R06,5,5,BMT,Astoria,36 Av,Q,False,N W,Elevated,40.756804,-73.929575,Astoria,Manhattan,0,0,0,NaN,POINT (-73.929575 40.756804)


In [7]:
metro_map.Borough.unique()

array(['Q', 'M', 'Bk', 'Bx', 'SI'], dtype=object)

In [8]:
manhattan_stops = metro_map[metro_map['Borough'] == 'M']
brooklyn_stops = metro_map[metro_map['Borough'] == 'Bk']
queens_stops = metro_map[metro_map['Borough'] == 'Q']
bronx_stops = metro_map[metro_map['Borough'] == 'Bx']
staten_island_stops = metro_map[metro_map['Borough'] == 'SI']

In [9]:
manhattan_stops.shape, brooklyn_stops.shape, queens_stops.shape, bronx_stops.shape, staten_island_stops.shape

((153, 19), (169, 19), (83, 19), (70, 19), (21, 19))

In [10]:
metro_map.Line.unique()

array(['Astoria', 'Broadway - Brighton', 'Broadway', 'Manhattan Bridge',
       '4th Av', 'Sea Beach / West End / Culver / Brighton', 'West End',
       'Sea Beach', 'Jamaica', 'Myrtle Av', 'Canarsie',
       'Franklin Shuttle', '8th Av - Fulton St', 'Concourse',
       '6th Av - Culver', 'Liberty Av', 'Rockaway', '63rd St',
       'Queens Blvd', 'Queens - Archer', 'Crosstown', 'Broadway - 7Av',
       'Clark St', 'Eastern Pky', 'Nostrand', 'Pelham', 'Jerome Av',
       'Lexington Av', 'Lenox - White Plains Rd', 'Dyre Av', 'Flushing',
       'Lexington - Shuttle', 'Second Av', 'Staten Island'], dtype=object)

In [11]:
metro_map.columns

Index(['GTFS Stop ID', 'Station ID', 'Complex ID', 'Division', 'Line',
       'Stop Name', 'Borough', 'CBD', 'Daytime Routes', 'Structure',
       'GTFS Latitude', 'GTFS Longitude', 'North Direction Label',
       'South Direction Label', 'ADA', 'ADA Northbound', 'ADA Southbound',
       'ADA Notes', 'Georeference'],
      dtype='object')

In [12]:
metro_map.rename(columns={'GTFS Latitude': 'latitude'}, inplace=True)
metro_map.rename(columns={'GTFS Longitude': 'longitude'}, inplace=True)

In [13]:
metro_map.to_csv('/content/drive/My Drive/X999/metro_map.csv', index=False)

# **1. Metro and SVI**

In [19]:
ny_svi_census_track = pd.read_csv('/content/drive/My Drive/X999/ny_svi_census_track.csv')
# ny_svi_counties = pd.read_csv('/content/drive/My Drive/X999/ny_svi_counties.csv')

In [20]:
ny_svi_counties.head()

,ST,STATE,ST_ABBR,STCNTY,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,M_HU,E_HH,M_HH,E_POV150,M_POV150,E_UNEMP,M_UNEMP,E_HBURD,M_HBURD,E_NOHSDP,M_NOHSDP,E_UNINSUR,M_UNINSUR,E_AGE65,M_AGE65,E_AGE17,M_AGE17,E_DISABL,M_DISABL,E_SNGPNT,M_SNGPNT,E_LIMENG,M_LIMENG,E_MINRTY,M_MINRTY,E_MUNIT,M_MUNIT,E_MOBILE,M_MOBILE,E_CROWD,M_CROWD,E_NOVEH,M_NOVEH,E_GROUPQ,M_GROUPQ,EP_POV150,MP_POV150,EP_UNEMP,MP_UNEMP,EP_HBURD,MP_HBURD,EP_NOHSDP,MP_NOHSDP,EP_UNINSUR,MP_UNINSUR,EP_AGE65,MP_AGE65,EP_AGE17,MP_AGE17,EP_DISABL,MP_DISABL,EP_SNGPNT,MP_SNGPNT,EP_LIMENG,MP_LIMENG,EP_MINRTY,MP_MINRTY,EP_MUNIT,MP_MUNIT,EP_MOBILE,MP_MOBILE,EP_CROWD,MP_CROWD,EP_NOVEH,MP_NOVEH,EP_GROUPQ,MP_GROUPQ,EPL_POV150,EPL_UNEMP,EPL_HBURD,EPL_NOHSDP,EPL_UNINSUR,SPL_THEME1,RPL_THEME1,EPL_AGE65,EPL_AGE17,EPL_DISABL,EPL_SNGPNT,EPL_LIMENG,SPL_THEME2,RPL_THEME2,EPL_MINRTY,SPL_THEME3,RPL_THEME3,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,F_POV150,F_UNEMP,F_HBURD,F_NOHSDP,F_UNINSUR,F_THEME1,F_AGE65,F_AGE17,F_DISABL,F_SNGPNT,F_LIMENG,F_THEME2,F_MINRTY,F_THEME3,F_MUNIT,F_MOBILE,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_DAYPOP,E_NOINT,M_NOINT,E_AFAM,M_AFAM,E_HISP,M_HISP,E_ASIAN,M_ASIAN,E_AIAN,M_AIAN,E_NHPI,M_NHPI,E_TWOMORE,M_TWOMORE,E_OTHERRACE,M_OTHERRACE,EP_NOINT,MP_NOINT,EP_AFAM,MP_AFAM,EP_HISP,MP_HISP,EP_ASIAN,MP_ASIAN,EP_AIAN,MP_AIAN,EP_NHPI,MP_NHPI,EP_TWOMORE,MP_TWOMORE,EP_OTHERRACE,MP_OTHERRACE
0,36,New York,NY,36001,Albany County,36001,"Albany County, New York",522.887676,315041,0,146137,122,132175,949,55161,2365,8915,876,34651,1502,15683,1236,9405,822,55160,51,57079,65,36721,1387,7998,669,5252,685,94930,640,19526,1030,1968,286,1925,399,16477,955,18083,156,18.4,0.8,5.2,0.5,26.2,1.1,7.4,0.6,3.0,0.3,17.5,0.1,18.1,0.1,11.8,0.4,6.0,0.5,1.8,0.2,30.1,0.2,13.4,0.7,1.3,0.2,1.4,0.3,12.5,0.7,5.7,0.0,0.2459,0.4590,0.7213,0.0820,0.0492,1.5574,0.1639,0.1967,0.2131,0.2131,0.7705,0.7213,2.1147,0.2951,0.7869,0.7869,0.7869,0.8852,0.2131,0.4262,0.8361,0.7213,3.0819,0.8361,7.5409,0.4754,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,412694,15442,937,35953,1120,20613,0,22902,556,152,69,177,49,14181,1293,952,309,11.7,0.7,11.4,0.4,6.5,0.0,7.3,0.2,0.0,0.1,0.1,0.1,4.5,0.4,0.3,0.1
1,36,New York,NY,36003,Allegany County,36003,"Allegany County, New York",1029.434838,47222,0,23636,54,16813,321,11314,579,1545,248,3475,271,2592,291,2491,365,9128,50,9265,47,7664,407,617,98,271,109,3027,59,462,114,3855,249,264,70,1225,161,4663,48,26.7,1.4,7.3,1.1,20.7,1.6,8.6,1.0,5.3,0.8,19.3,0.1,19.6,0.1,16.4,0.9,3.6,0.6,0.6,0.2,6.4,0.1,1.9,0.5,16.3,1.1,1.6,0.4,7.3,0.9,9.9,0.1,0.8852,0.9508,0.0984,0.3770,0.7377,3.0491,0.6885,0.5410,0.4262,0.8033,0.0820,0.3115,2.1640,0.3443,0.0328,0.0328,0.0328,0.0328,0.9672,0.5410,0.2623,0.9836,2.7869,0.6721,8.0328,0.5246,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,2,3,44037,2691,234,722,82,886,0,439,149,51,31,26,25,865,183,38,22,16.0,1.4,1.5,0.2,1.9,0.0,0.9,0.3,0.1,0.1,0.1,0.1,1.8,0.4,0.1,0.1
2,36,New York,NY,36005,Bronx County,36005,"Bronx County, New York",42.176112,1443229,0,547816,514,525387,1329,544179,10015,73599,2785,254627,5427,240495,4254,108251,3744,191647,185,355908,28,225484,4154,74886,2395,195143,4517,1319327,758,369720,3322,828,293,65669,2616,316165,3062,53830,1280,38.6,0.7,11.2,0.4,48.5,1.0,25.5,0.4,7.6,0.3,13.3,0.1,24.7,0.1,15.8,0.3,14.3,0.5,14.5,0.3,91.4,0.1,67.5,0.6,0.2,0.1,12.5,0.5,60.2,0.6,3.7,0.1,1.0000,1.0000,1.0000,1.0000,0.9508,4.9508,1.0000,0.0000,0.9672,0.7049,1.0000,0.9836,3.6557,0.9672,1.0000,1.0000,1.0000,0.9836,0.0328,1.0000,0.9836,0.4426,3.4426,0.9508,13.0491,1.0000,1,1,1,1,1,5,0,1,0,1,1,3,1,1,1,0,1,1,0,3,12,1153071,85216,2779,408777,1935,814017,0,54493,950,3225,415,427,298,25335,1891,13053,1478,16.2,0.5,28.3,0.1,56.4,0.0,3.8,0.1,0.2,0.1,0.0,0.1,1.8,0.1,0.9,0.1
3,36,New York,NY,36007,Broome County,36007,"Broome County, New York",705.650281,198365,0,92596,87,81339,810,51702,2158,6514,798,22491,1132,12353,963,7375,905,39154,36,37879,0,31060,1333,4626,528,2769,450,37103,328,7799,620,3703,388,1540,313,10204,768,

In [21]:
ny_svi_counties.COUNTY.unique()

array(['Albany County', 'Allegany County', 'Bronx County',
       'Broome County', 'Cattaraugus County', 'Cayuga County',
       'Chautauqua County', 'Chemung County', 'Chenango County',
       'Clinton County', 'Columbia County', 'Cortland County',
       'Delaware County', 'Dutchess County', 'Erie County',
       'Essex County', 'Franklin County', 'Fulton County',
       'Genesee County', 'Greene County', 'Hamilton County',
       'Herkimer County', 'Jefferson County', 'Kings County',
       'Lewis County', 'Livingston County', 'Madison County',
       'Monroe County', 'Montgomery County', 'Nassau County',
       'New York County', 'Niagara County', 'Oneida County',
       'Onondaga County', 'Ontario County', 'Orange County',
       'Orleans County', 'Oswego County', 'Otsego County',
       'Putnam County', 'Queens County', 'Rensselaer County',
       'Richmond County', 'Rockland County', 'St. Lawrence County',
       'Saratoga County', 'Schenectady County', 'Schoharie County',
     

In [27]:
brooklyn_svi_df = ny_svi_counties[ny_svi_counties['COUNTY'] == 'Kings County']
manhattan_svi_df = ny_svi_counties[ny_svi_counties['COUNTY'] == 'New York County']
queens_svi_df = ny_svi_counties[ny_svi_counties['COUNTY'] == 'Queens County']
bronx_svi_df = ny_svi_counties[ny_svi_counties['COUNTY'] == 'Bronx County']
staten_island_svi_df = ny_svi_counties[ny_svi_counties['COUNTY'] == 'Richmond County']

In [28]:
brooklyn_svi_df.shape, manhattan_svi_df.shape, queens_svi_df.shape, bronx_svi_df.shape, staten_island_svi_df.shape

((1, 158), (1, 158), (1, 158), (1, 158), (1, 158))

In [30]:
brooklyn_svi_df

,ST,STATE,ST_ABBR,STCNTY,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,M_HU,E_HH,M_HH,E_POV150,M_POV150,E_UNEMP,M_UNEMP,E_HBURD,M_HBURD,E_NOHSDP,M_NOHSDP,E_UNINSUR,M_UNINSUR,E_AGE65,M_AGE65,E_AGE17,M_AGE17,E_DISABL,M_DISABL,E_SNGPNT,M_SNGPNT,E_LIMENG,M_LIMENG,E_MINRTY,M_MINRTY,E_MUNIT,M_MUNIT,E_MOBILE,M_MOBILE,E_CROWD,M_CROWD,E_NOVEH,M_NOVEH,E_GROUPQ,M_GROUPQ,EP_POV150,MP_POV150,EP_UNEMP,MP_UNEMP,EP_HBURD,MP_HBURD,EP_NOHSDP,MP_NOHSDP,EP_UNINSUR,MP_UNINSUR,EP_AGE65,MP_AGE65,EP_AGE17,MP_AGE17,EP_DISABL,MP_DISABL,EP_SNGPNT,MP_SNGPNT,EP_LIMENG,MP_LIMENG,EP_MINRTY,MP_MINRTY,EP_MUNIT,MP_MUNIT,EP_MOBILE,MP_MOBILE,EP_CROWD,MP_CROWD,EP_NOVEH,MP_NOVEH,EP_GROUPQ,MP_GROUPQ,EPL_POV150,EPL_UNEMP,EPL_HBURD,EPL_NOHSDP,EPL_UNINSUR,SPL_THEME1,RPL_THEME1,EPL_AGE65,EPL_AGE17,EPL_DISABL,EPL_SNGPNT,EPL_LIMENG,SPL_THEME2,RPL_THEME2,EPL_MINRTY,SPL_THEME3,RPL_THEME3,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,F_POV150,F_UNEMP,F_HBURD,F_NOHSDP,F_UNINSUR,F_THEME1,F_AGE65,F_AGE17,F_DISABL,F_SNGPNT,F_LIMENG,F_THEME2,F_MINRTY,F_THEME3,F_MUNIT,F_MOBILE,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_DAYPOP,E_NOINT,M_NOINT,E_AFAM,M_AFAM,E_HISP,M_HISP,E_ASIAN,M_ASIAN,E_AIAN,M_AIAN,E_NHPI,M_NHPI,E_TWOMORE,M_TWOMORE,E_OTHERRACE,M_OTHERRACE,EP_NOINT,MP_NOINT,EP_AFAM,MP_AFAM,EP_HISP,MP_HISP,EP_ASIAN,MP_ASIAN,EP_AIAN,MP_AIAN,EP_NHPI,MP_NHPI,EP_TWOMORE,MP_TWOMORE,EP_OTHERRACE,MP_OTHERRACE
23,36,New York,NY,36047,Kings County,36047,"Kings County, New York",69.376539,2679620,0,1079551,304,997957,2140,750523,9987,98832,3315,367269,5229,301813,4294,168139,4614,388584,61,604364,0,273681,4156,69111,2050,281988,5395,1713006,1742,467173,3787,1495,294,102369,2598,552059,4312,47910,458,28.3,0.4,7.3,0.2,36.8,0.5,16.2,0.2,6.3,0.2,14.5,0.1,22.6,0.0,10.3,0.2,7.0,0.2,11.3,0.2,63.9,0.1,43.3,0.4,0.1,0.1,10.3,0.3,55.3,0.4,1.8,0.0,0.9836,0.9508,0.9836,0.9672,0.918,4.8032,0.9836,0.0492,0.9016,0.0984,0.9672,0.9672,2.9836,0.8033,0.9672,0.9672,0.9672,0.9672,0.0,0.9836,0.9672,0.1475,3.0655,0.8033,11.8195,0.9672,1,1,1,1,1,5,0,1,0,1,1,3,1,1,1,0,1,1,0,3,12,2282064,135747,2912,759133,3619,506473,0,319099,2541,3617,501,1096,283,101404,4674,22184,2594,13.6,0.3,28.3,0.1,18.9,0.0,11.9,0.1,0.1,0.1,0.0,0.1,3.8,0.2,0.8,0.1


In [31]:
manhattan_svi_df

,ST,STATE,ST_ABBR,STCNTY,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,M_HU,E_HH,M_HH,E_POV150,M_POV150,E_UNEMP,M_UNEMP,E_HBURD,M_HBURD,E_NOHSDP,M_NOHSDP,E_UNINSUR,M_UNINSUR,E_AGE65,M_AGE65,E_AGE17,M_AGE17,E_DISABL,M_DISABL,E_SNGPNT,M_SNGPNT,E_LIMENG,M_LIMENG,E_MINRTY,M_MINRTY,E_MUNIT,M_MUNIT,E_MOBILE,M_MOBILE,E_CROWD,M_CROWD,E_NOVEH,M_NOVEH,E_GROUPQ,M_GROUPQ,EP_POV150,MP_POV150,EP_UNEMP,MP_UNEMP,EP_HBURD,MP_HBURD,EP_NOHSDP,MP_NOHSDP,EP_UNINSUR,MP_UNINSUR,EP_AGE65,MP_AGE65,EP_AGE17,MP_AGE17,EP_DISABL,MP_DISABL,EP_SNGPNT,MP_SNGPNT,EP_LIMENG,MP_LIMENG,EP_MINRTY,MP_MINRTY,EP_MUNIT,MP_MUNIT,EP_MOBILE,MP_MOBILE,EP_CROWD,MP_CROWD,EP_NOVEH,MP_NOVEH,EP_GROUPQ,MP_GROUPQ,EPL_POV150,EPL_UNEMP,EPL_HBURD,EPL_NOHSDP,EPL_UNINSUR,SPL_THEME1,RPL_THEME1,EPL_AGE65,EPL_AGE17,EPL_DISABL,EPL_SNGPNT,EPL_LIMENG,SPL_THEME2,RPL_THEME2,EPL_MINRTY,SPL_THEME3,RPL_THEME3,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,F_POV150,F_UNEMP,F_HBURD,F_NOHSDP,F_UNINSUR,F_THEME1,F_AGE65,F_AGE17,F_DISABL,F_SNGPNT,F_LIMENG,F_THEME2,F_MINRTY,F_THEME3,F_MUNIT,F_MOBILE,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_DAYPOP,E_NOINT,M_NOINT,E_AFAM,M_AFAM,E_HISP,M_HISP,E_ASIAN,M_ASIAN,E_AIAN,M_AIAN,E_NHPI,M_NHPI,E_TWOMORE,M_TWOMORE,E_OTHERRACE,M_OTHERRACE,EP_NOINT,MP_NOINT,EP_AFAM,MP_AFAM,EP_HISP,MP_HISP,EP_ASIAN,MP_ASIAN,EP_AIAN,MP_AIAN,EP_NHPI,MP_NHPI,EP_TWOMORE,MP_TWOMORE,EP_OTHERRACE,MP_OTHERRACE
30,36,New York,NY,36061,New York County,36061,"New York County, New York",22.657963,1645867,0,913058,441,772601,4457,355158,8027,64535,2909,222057,4886,146231,4273,74860,3930,284256,103,235675,143,181813,4558,36720,2054,126474,4384,896067,1647,818482,5078,522,199,42068,2482,601621,5689,68812,851,22.2,0.5,6.8,0.3,28.7,0.6,11.5,0.3,4.6,0.2,17.3,0.1,14.3,0.1,11.1,0.3,4.8,0.3,8.0,0.3,54.4,0.1,89.7,0.6,0.1,0.1,5.4,0.3,77.9,0.5,4.2,0.1,0.6393,0.8689,0.8689,0.7541,0.623,3.7542,0.9016,0.1639,0.0164,0.1311,0.4098,0.9508,1.672,0.0984,0.9508,0.9508,0.9508,1.0,0.0,0.9344,1.0,0.5246,3.459,0.9836,9.836,0.8525,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,1,1,0,3,5,3406394,80193,2665,203409,2371,428297,0,197451,1972,1637,407,471,215,53491,3135,11311,2172,10.4,0.3,12.4,0.1,26.0,0.0,12.0,0.1,0.1,0.1,0.0,0.1,3.3,0.2,0.7,0.1


In [32]:
queens_svi_df

,ST,STATE,ST_ABBR,STCNTY,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,M_HU,E_HH,M_HH,E_POV150,M_POV150,E_UNEMP,M_UNEMP,E_HBURD,M_HBURD,E_NOHSDP,M_NOHSDP,E_UNINSUR,M_UNINSUR,E_AGE65,M_AGE65,E_AGE17,M_AGE17,E_DISABL,M_DISABL,E_SNGPNT,M_SNGPNT,E_LIMENG,M_LIMENG,E_MINRTY,M_MINRTY,E_MUNIT,M_MUNIT,E_MOBILE,M_MOBILE,E_CROWD,M_CROWD,E_NOVEH,M_NOVEH,E_GROUPQ,M_GROUPQ,EP_POV150,MP_POV150,EP_UNEMP,MP_UNEMP,EP_HBURD,MP_HBURD,EP_NOHSDP,MP_NOHSDP,EP_UNINSUR,MP_UNINSUR,EP_AGE65,MP_AGE65,EP_AGE17,MP_AGE17,EP_DISABL,MP_DISABL,EP_SNGPNT,MP_SNGPNT,EP_LIMENG,MP_LIMENG,EP_MINRTY,MP_MINRTY,EP_MUNIT,MP_MUNIT,EP_MOBILE,MP_MOBILE,EP_CROWD,MP_CROWD,EP_NOVEH,MP_NOVEH,EP_GROUPQ,MP_GROUPQ,EPL_POV150,EPL_UNEMP,EPL_HBURD,EPL_NOHSDP,EPL_UNINSUR,SPL_THEME1,RPL_THEME1,EPL_AGE65,EPL_AGE17,EPL_DISABL,EPL_SNGPNT,EPL_LIMENG,SPL_THEME2,RPL_THEME2,EPL_MINRTY,SPL_THEME3,RPL_THEME3,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,F_POV150,F_UNEMP,F_HBURD,F_NOHSDP,F_UNINSUR,F_THEME1,F_AGE65,F_AGE17,F_DISABL,F_SNGPNT,F_LIMENG,F_THEME2,F_MINRTY,F_THEME3,F_MUNIT,F_MOBILE,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_DAYPOP,E_NOINT,M_NOINT,E_AFAM,M_AFAM,E_HISP,M_HISP,E_ASIAN,M_ASIAN,E_AIAN,M_AIAN,E_NHPI,M_NHPI,E_TWOMORE,M_TWOMORE,E_OTHERRACE,M_OTHERRACE,EP_NOINT,MP_NOINT,EP_AFAM,MP_AFAM,EP_HISP,MP_HISP,EP_ASIAN,MP_ASIAN,EP_AIAN,MP_AIAN,EP_NHPI,MP_NHPI,EP_TWOMORE,MP_TWOMORE,EP_OTHERRACE,MP_OTHERRACE
40,36,New York,NY,36081,Queens County,36081,"Queens County, New York",108.724025,2360826,0,896825,352,816859,2540,467135,8682,86855,3056,276026,4573,294227,4341,204984,5109,389399,148,469866,0,233084,4100,42953,1982,323460,6389,1799907,1614,323202,3931,2025,448,80249,2330,300112,3427,31917,220,20.0,0.4,7.0,0.2,33.8,0.5,17.2,0.3,8.7,0.2,16.5,0.1,19.9,0.0,9.9,0.2,5.2,0.2,14.5,0.3,76.2,0.1,36.0,0.4,0.2,0.1,9.8,0.3,36.7,0.4,1.4,0.0,0.377,0.9016,0.9672,0.9836,0.9836,4.213,0.9508,0.0984,0.4754,0.0328,0.5574,0.9836,2.1476,0.3279,0.9836,0.9836,0.9836,0.9508,0.0328,0.9672,0.9508,0.0164,2.918,0.7213,10.2622,0.8852,0,1,1,1,1,4,0,0,0,0,1,1,1,1,1,0,1,1,0,3,9,1962559,87470,2339,394824,3206,661807,0,611350,2832,6251,735,1048,331,75045,3553,49582,3370,10.7,0.3,16.7,0.1,28.0,0.0,25.9,0.1,0.3,0.1,0.0,0.1,3.2,0.2,2.1,0.1


In [33]:
bronx_svi_df

,ST,STATE,ST_ABBR,STCNTY,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,M_HU,E_HH,M_HH,E_POV150,M_POV150,E_UNEMP,M_UNEMP,E_HBURD,M_HBURD,E_NOHSDP,M_NOHSDP,E_UNINSUR,M_UNINSUR,E_AGE65,M_AGE65,E_AGE17,M_AGE17,E_DISABL,M_DISABL,E_SNGPNT,M_SNGPNT,E_LIMENG,M_LIMENG,E_MINRTY,M_MINRTY,E_MUNIT,M_MUNIT,E_MOBILE,M_MOBILE,E_CROWD,M_CROWD,E_NOVEH,M_NOVEH,E_GROUPQ,M_GROUPQ,EP_POV150,MP_POV150,EP_UNEMP,MP_UNEMP,EP_HBURD,MP_HBURD,EP_NOHSDP,MP_NOHSDP,EP_UNINSUR,MP_UNINSUR,EP_AGE65,MP_AGE65,EP_AGE17,MP_AGE17,EP_DISABL,MP_DISABL,EP_SNGPNT,MP_SNGPNT,EP_LIMENG,MP_LIMENG,EP_MINRTY,MP_MINRTY,EP_MUNIT,MP_MUNIT,EP_MOBILE,MP_MOBILE,EP_CROWD,MP_CROWD,EP_NOVEH,MP_NOVEH,EP_GROUPQ,MP_GROUPQ,EPL_POV150,EPL_UNEMP,EPL_HBURD,EPL_NOHSDP,EPL_UNINSUR,SPL_THEME1,RPL_THEME1,EPL_AGE65,EPL_AGE17,EPL_DISABL,EPL_SNGPNT,EPL_LIMENG,SPL_THEME2,RPL_THEME2,EPL_MINRTY,SPL_THEME3,RPL_THEME3,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,F_POV150,F_UNEMP,F_HBURD,F_NOHSDP,F_UNINSUR,F_THEME1,F_AGE65,F_AGE17,F_DISABL,F_SNGPNT,F_LIMENG,F_THEME2,F_MINRTY,F_THEME3,F_MUNIT,F_MOBILE,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_DAYPOP,E_NOINT,M_NOINT,E_AFAM,M_AFAM,E_HISP,M_HISP,E_ASIAN,M_ASIAN,E_AIAN,M_AIAN,E_NHPI,M_NHPI,E_TWOMORE,M_TWOMORE,E_OTHERRACE,M_OTHERRACE,EP_NOINT,MP_NOINT,EP_AFAM,MP_AFAM,EP_HISP,MP_HISP,EP_ASIAN,MP_ASIAN,EP_AIAN,MP_AIAN,EP_NHPI,MP_NHPI,EP_TWOMORE,MP_TWOMORE,EP_OTHERRACE,MP_OTHERRACE
2,36,New York,NY,36005,Bronx County,36005,"Bronx County, New York",42.176112,1443229,0,547816,514,525387,1329,544179,10015,73599,2785,254627,5427,240495,4254,108251,3744,191647,185,355908,28,225484,4154,74886,2395,195143,4517,1319327,758,369720,3322,828,293,65669,2616,316165,3062,53830,1280,38.6,0.7,11.2,0.4,48.5,1.0,25.5,0.4,7.6,0.3,13.3,0.1,24.7,0.1,15.8,0.3,14.3,0.5,14.5,0.3,91.4,0.1,67.5,0.6,0.2,0.1,12.5,0.5,60.2,0.6,3.7,0.1,1.0,1.0,1.0,1.0,0.9508,4.9508,1.0,0.0,0.9672,0.7049,1.0,0.9836,3.6557,0.9672,1.0,1.0,1.0,0.9836,0.0328,1.0,0.9836,0.4426,3.4426,0.9508,13.0491,1.0,1,1,1,1,1,5,0,1,0,1,1,3,1,1,1,0,1,1,0,3,12,1153071,85216,2779,408777,1935,814017,0,54493,950,3225,415,427,298,25335,1891,13053,1478,16.2,0.5,28.3,0.1,56.4,0.0,3.8,0.1,0.2,0.1,0.0,0.1,1.8,0.1,0.9,0.1


In [34]:
staten_island_svi_df

,ST,STATE,ST_ABBR,STCNTY,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,M_HU,E_HH,M_HH,E_POV150,M_POV150,E_UNEMP,M_UNEMP,E_HBURD,M_HBURD,E_NOHSDP,M_NOHSDP,E_UNINSUR,M_UNINSUR,E_AGE65,M_AGE65,E_AGE17,M_AGE17,E_DISABL,M_DISABL,E_SNGPNT,M_SNGPNT,E_LIMENG,M_LIMENG,E_MINRTY,M_MINRTY,E_MUNIT,M_MUNIT,E_MOBILE,M_MOBILE,E_CROWD,M_CROWD,E_NOVEH,M_NOVEH,E_GROUPQ,M_GROUPQ,EP_POV150,MP_POV150,EP_UNEMP,MP_UNEMP,EP_HBURD,MP_HBURD,EP_NOHSDP,MP_NOHSDP,EP_UNINSUR,MP_UNINSUR,EP_AGE65,MP_AGE65,EP_AGE17,MP_AGE17,EP_DISABL,MP_DISABL,EP_SNGPNT,MP_SNGPNT,EP_LIMENG,MP_LIMENG,EP_MINRTY,MP_MINRTY,EP_MUNIT,MP_MUNIT,EP_MOBILE,MP_MOBILE,EP_CROWD,MP_CROWD,EP_NOVEH,MP_NOVEH,EP_GROUPQ,MP_GROUPQ,EPL_POV150,EPL_UNEMP,EPL_HBURD,EPL_NOHSDP,EPL_UNINSUR,SPL_THEME1,RPL_THEME1,EPL_AGE65,EPL_AGE17,EPL_DISABL,EPL_SNGPNT,EPL_LIMENG,SPL_THEME2,RPL_THEME2,EPL_MINRTY,SPL_THEME3,RPL_THEME3,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,F_POV150,F_UNEMP,F_HBURD,F_NOHSDP,F_UNINSUR,F_THEME1,F_AGE65,F_AGE17,F_DISABL,F_SNGPNT,F_LIMENG,F_THEME2,F_MINRTY,F_THEME3,F_MUNIT,F_MOBILE,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_DAYPOP,E_NOINT,M_NOINT,E_AFAM,M_AFAM,E_HISP,M_HISP,E_ASIAN,M_ASIAN,E_AIAN,M_AIAN,E_NHPI,M_NHPI,E_TWOMORE,M_TWOMORE,E_OTHERRACE,M_OTHERRACE,EP_NOINT,MP_NOINT,EP_AFAM,MP_AFAM,EP_HISP,MP_HISP,EP_ASIAN,MP_ASIAN,EP_AIAN,MP_AIAN,EP_NHPI,MP_NHPI,EP_TWOMORE,MP_TWOMORE,EP_OTHERRACE,MP_OTHERRACE
42,36,New York,NY,36085,Richmond County,36085,"Richmond County, New York",57.522511,492925,0,183524,106,170000,956,80598,3667,12999,1085,47268,2116,39031,2028,19179,1439,81753,96,106747,137,50548,2009,9290,839,27472,1496,207022,516,21255,1266,490,211,8934,838,25884,1233,7087,112,16.5,0.8,5.4,0.4,27.8,1.2,11.3,0.6,3.9,0.3,16.6,0.1,21.7,0.1,10.3,0.4,5.5,0.5,5.9,0.3,42.0,0.1,11.6,0.7,0.3,0.1,5.3,0.5,15.2,0.7,1.4,0.0,0.1148,0.541,0.8197,0.7213,0.3607,2.5575,0.541,0.1311,0.8525,0.0984,0.6557,0.918,2.6557,0.6393,0.9016,0.9016,0.9016,0.8525,0.0984,0.918,0.9344,0.0164,2.8197,0.6885,8.9345,0.7049,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,1,1,0,2,4,407413,19694,1243,44626,900,92411,0,56162,846,692,161,14,23,11157,1259,1960,569,11.6,0.7,9.1,0.2,18.7,0.0,11.4,0.2,0.1,0.1,0.0,0.1,2.3,0.3,0.4,0.1


In [97]:
# rank svi score
man_svi = manhattan_svi_df['RPL_THEMES'].sum()
bk_svi = brooklyn_svi_df['RPL_THEMES'].sum()
queens_svi = queens_svi_df['RPL_THEMES'].sum()
bx_svi = bronx_svi_df['RPL_THEMES'].sum()
si_svi = staten_island_svi_df['RPL_THEMES'].sum()
man_svi, bk_svi, queens_svi, bx_svi, si_svi

(np.float64(0.8525),
 np.float64(0.9672),
 np.float64(0.8852),
 np.float64(1.0),
 np.float64(0.7049))

In [90]:
 # housing and transportations
 man_ht_svi = manhattan_svi_df['RPL_THEME4'].sum()
 bk_ht_svi = brooklyn_svi_df['RPL_THEME4'].sum()
 queens_ht_svi = queens_svi_df['RPL_THEME4'].sum()
 bx_ht_svi = bronx_svi_df['RPL_THEME4'].sum()
 si_ht_svi = staten_island_svi_df['RPL_THEME4'].sum()
 man_ht_svi, bk_ht_svi, queens_ht_svi, bx_ht_svi, si_ht_svi

(np.float64(0.9836),
 np.float64(0.8033),
 np.float64(0.7213),
 np.float64(0.9508),
 np.float64(0.6885))

In [85]:
# area
man_area = manhattan_svi_df['AREA_SQMI'].sum()
bk_area = brooklyn_svi_df['AREA_SQMI'].sum()
queens_area = queens_svi_df['AREA_SQMI'].sum()
bx_area = bronx_svi_df['AREA_SQMI'].sum()
si_area = staten_island_svi_df['AREA_SQMI'].sum()
man_area, bk_area, queens_area, bx_area, si_area

(np.float64(22.65796343576),
 np.float64(69.376538641368),
 np.float64(108.724025120896),
 np.float64(42.176112202748),
 np.float64(57.52251071336))

In [40]:
brooklyn_pop = brooklyn_svi_df['E_TOTPOP'].astype(int)
manhattan_pop = manhattan_svi_df['E_TOTPOP'].astype(int)
queens_pop = queens_svi_df['E_TOTPOP'].astype(int)
bronx_pop = bronx_svi_df['E_TOTPOP'].astype(int)
staten_island_pop = staten_island_svi_df['E_TOTPOP'].astype(int)

In [41]:
brooklyn_pop, manhattan_pop, queens_pop, bronx_pop, staten_island_pop

(23    2679620
 Name: E_TOTPOP, dtype: int64,
 30    1645867
 Name: E_TOTPOP, dtype: int64,
 40    2360826
 Name: E_TOTPOP, dtype: int64,
 2    1443229
 Name: E_TOTPOP, dtype: int64,
 42    492925
 Name: E_TOTPOP, dtype: int64)

In [50]:
print(manhattan_stops.shape[0], manhattan_stops['GTFS Stop ID'].nunique())
print(brooklyn_stops.shape[0], brooklyn_stops['GTFS Stop ID'].nunique())
print(queens_stops.shape[0], queens_stops['GTFS Stop ID'].nunique())
print(bronx_stops.shape[0], bronx_stops['GTFS Stop ID'].nunique())
print(staten_island_stops.shape[0], staten_island_stops['GTFS Stop ID'].nunique())
manhattan_metro_stop_count = manhattan_stops.shape[0]
brookly_metro_stop_count = brooklyn_stops.shape[0]
queens_metro_stop_count = queens_stops.shape[0]
bronx_metro_stop_count = bronx_stops.shape[0]
staten_island_metro_stop_count = staten_island_stops.shape[0]
manhattan_metro_stop_count, brookly_metro_stop_count, queens_metro_stop_count, bronx_metro_stop_count, staten_island_metro_stop_count
# good

153 153
169 169
83 83
70 70
21 21


(153, 169, 83, 70, 21)

In [60]:
manhattan_pop = manhattan_pop.sum()
brooklyn_pop = brooklyn_pop.sum()
queens_pop = queens_pop.sum()
bronx_pop = bronx_pop.sum()
staten_island_pop = staten_island_pop.sum()
manhattan_pop, brooklyn_pop, queens_pop, bronx_pop, staten_island_pop

(np.int64(1645867),
 np.int64(2679620),
 np.int64(2360826),
 np.int64(1443229),
 np.int64(492925))

In [64]:
man_stops_ratio = manhattan_metro_stop_count / manhattan_pop*1000
bk_stops_ratio = brookly_metro_stop_count / brooklyn_pop*1000
queens_stops_ratio = queens_metro_stop_count / queens_pop*1000
bronx_stops_ratio = bronx_metro_stop_count / bronx_pop*1000
si_stops_ratio = staten_island_metro_stop_count / staten_island_pop*1000
man_stops_ratio, bk_stops_ratio, queens_stops_ratio, bronx_stops_ratio, si_stops_ratio
# a better ratio would be walking buffer, see arcgis

(np.float64(0.09296012375240527),
 np.float64(0.0630686440614714),
 np.float64(0.03515718651014518),
 np.float64(0.048502351324703145),
 np.float64(0.04260283004513871))

# **2. Bus and SVI**

In [71]:
bus_stop_link = '/content/drive/My Drive/X999/Bus_Stop_Shelter.csv'
bus_df = pd.read_csv(bus_stop_link)

In [75]:
man_bus_stop = bus_df[bus_df['BoroName'] == 'Manhattan']
bk_bus_stop = bus_df[bus_df['BoroName'] == 'Brooklyn']
queens_bus_stop = bus_df[bus_df['BoroName'] == 'Queens']
bx_bus_stop = bus_df[bus_df['BoroName'] == 'Bronx']
si_bus_stop = bus_df[bus_df['BoroName'] == 'Staten Island']

In [81]:
man_bus_stop.drop_duplicates('the_geom', inplace=True)
bk_bus_stop.drop_duplicates('the_geom', inplace=True)
queens_bus_stop.drop_duplicates('the_geom', inplace=True)
bx_bus_stop.drop_duplicates('the_geom', inplace=True)
si_bus_stop.drop_duplicates('the_geom', inplace=True)
man_bus_stop.shape, bk_bus_stop.shape, queens_bus_stop.shape, bx_bus_stop.shape, si_bus_stop.shape

((720, 18), (871, 18), (968, 18), (563, 18), (208, 18))

In [76]:
man_bus_stop.shape, bk_bus_stop.shape, queens_bus_stop.shape, bx_bus_stop.shape, si_bus_stop.shape

((720, 18), (871, 18), (968, 18), (563, 18), (208, 18))

In [91]:
man_bus_stop_ratio = man_bus_stop.shape[0] / manhattan_pop*1000
bk_bus_stop_ratio = bk_bus_stop.shape[0] / brooklyn_pop*1000
queens_bus_stop_ratio = queens_bus_stop.shape[0] / queens_pop*1000
bx_bus_stop_ratio = bx_bus_stop.shape[0] / bronx_pop*1000
si_bus_stop_ratio = si_bus_stop.shape[0] / staten_island_pop*1000

In [77]:
manhattan_metro_stop_count, brookly_metro_stop_count, queens_metro_stop_count, bronx_metro_stop_count, staten_island_metro_stop_count

(153, 169, 83, 70, 21)

# **3. Park Properties**

In [102]:
park_link = '/content/drive/My Drive/X999/Parks_Properties.csv'
park_df = pd.read_csv(park_link)

In [103]:
park_df.head()

,ACQUISITIONDATE,ACRES,ADDRESS,BOROUGH,CLASS,COMMUNITYBOARD,COUNCILDISTRICT,DEPARTMENT,EAPPLY,GISOBJID,GISPROPNUM,GlobalID,JURISDICTION,LOCATION,MAPPED,NAME311,NYS_ASSEMBLY,NYS_SENATE,OBJECTID,OMPPROPID,PARENTID,PERMIT,PERMITDISTRICT,PERMITPARENT,PIP_RATABLE,PRECINCT,RETIRED,SIGNNAME,SUBCATEGORY,TYPECATEGORY,US_CONGRESS,WATERFRONT,ZIPCODE,multipolygon
0,01/27/1916 12:00:00 AM,0.54500000,4901 BROADWAY,M,PARK,112,10,M-12,Dyckman House Museum,100003823.00000000,M026,EF9ED529-837F-4538-9593-5E995E8C6A16,DPR,Broadway and W. 204 St.,True,Dyckman House Museum,72.00000000,31.00000000,170894,M026,M-12,False,M-12,M-12,False,34.00000000,False,Dyckman House Museum,Historic House,Historic House Park,13.00000000,False,10034,"MULTIPOLYGON (((-73.92305320407046 40.867183685064035, -73.92248480546031 40.86738648923204, -73.922723681874 40.867760801830364, -73.922963959853 40.86767507473253, -73.92292213801646 40.86760954251678, -73.92325282051638 40.867491580922675, -73.92305320407046 40.867183685064035)))"
1,11/20/2002 12:00:00 AM,0.11400000,851 HORNADAY PLACE,X,PARK,206,15,X-06,Volky Garden & Flowers,100005124.00000000,X346,9461E04B-228E-48BD-B495-BB50A192CA5E,DPR,Hornaday Pl bet. Crotona Pkwy and Mohegan Av,False,Volky Garden & Flowers,87.00000000,32.00000000,170872,X346,X-06,True,X-06,X-06,False,48.00000000,False,Volky Garden & Flowers,Greenthumb,Garden,15.00000000,False,10460,"MULTIPOLYGON (((-73.88234391814649 40.847073877491205, -73.88247978327146 40.84716685050586, -73.8827266880881 40.846960571924534, -73.88258891934521 40.84686919023183, -73.88234391814649 40.847073877491205)))"
2,01/19/2006 12:00:00 AM,6.21900000,78-01 57 AVENUE,Q,PARK,404,30,Q-04,Elmhurst Park,100000211.00000000,Q492,21259B28-434B-40F8-A830-D58F80EF3022,DPR,"Grand Ave., 57 Ave. bet. 74 St. and 80 St.",False,Elmhurst Park,30.00000000,12.00000000,170871,Q492,Q-04,True,Q-04,Q-04,True,110.00000000,False,Elmhurst Park,Neighborhood Park,Community Park,6.00000000,False,11373,"MULTIPOLYGON (((-73.88436854859785 40.72890922012064, -73.88410206116812 40.72892961439369, -73.8848518790976 40.730508774871076, -73.88461344791205 40.73057617884767, -73.88500422955502 40.731336373465595, -73.88527230317742 40.73095085014654, -73.88600993196299 40.73060573855411, -73.88592081033991 40.73020249052168, -73.88594238075623 40.73010217985134, -73.88540439640101 40.7288299383273, -73.88436854859785 40.72890922012064)))"
3,07/10/1902 12:00:00 AM,3.31200000,76-01 FDR DRIVE,M,PARK,108,5,M-08,John Jay Park,100004745.00000000,M045,8A8EC925-61FD-4C3B-9C7B-E9053A03A658,DPR,"FDR Dr., E 76 St. To E 78 St.",True,John Jay Park,76.00000000,28.00000000,170875,M045,M-08,True,M-08,M-08,True,19.00000000,False,John Jay Park,Neighborhood Plgd,Community Park,12.00000000,False,10021,"MULTIPOLYGON (((-73.94903734207423 40.76900910002715, -73.94851239993555 40.769588905309284, -73.94945658992339 40.76999073451401, -73.94992274087893 40.76935197587993, -73.9501148233556 40.769433720469046, -73.95046972076757 40.76894733059399, -73.9494078091686 40.7684954070984, -73.94903734207423 40.76900910002715)))"
4,09/04/1839 12:00:00 AM,20.16500000,18 MT MORRIS PARK WEST,M,PARK,111,9,M-11,Marcus Garvey Park,100004516.00000000,M058,9086E0B4-3DFA-4286-9C40-9103CCB0D98B,DPR,"Madison Ave, E. 120 St. to E. 124 St.",False,Marcus Garvey Park,68.00000000,30.00000000,170863,M058,M-11,True,M-11,M-11,False,25.00000000,False,Marcus Garvey Park,Large Park,Community Park,13.00000000,False,10027,"MULTIPOLYGON (((-73.94591164848372 40.80390727753462, -73.94313133933686 40.802730471482725, -73.94142727273162 40.80506372284657, -73.9442076503982 40.80624056955785, -73.94591164848372 40.80390727753462)))"


# **4. Put everything in a dataframe**

In [98]:
data = {
    'Borough': ['Manhattan', 'Brooklyn', 'Queens', 'Bronx', 'Staten Island'],
    'Population': [manhattan_pop, brooklyn_pop, queens_pop, bronx_pop, staten_island_pop],
    'Area': [man_area, bk_area, queens_area, bx_area, si_area],
    'Train Stations': [manhattan_metro_stop_count, brookly_metro_stop_count, queens_metro_stop_count, bronx_metro_stop_count, staten_island_metro_stop_count],
    'Bus Stops': [man_bus_stop.shape[0], bk_bus_stop.shape[0], queens_bus_stop.shape[0], bx_bus_stop.shape[0], si_bus_stop.shape[0]],
    'Stops Ratio': [man_stops_ratio, bk_stops_ratio, queens_stops_ratio, bronx_stops_ratio, si_stops_ratio],
    'Bus Stops Ratio': [man_bus_stop_ratio, bk_bus_stop_ratio, queens_bus_stop_ratio, bx_bus_stop_ratio, si_bus_stop_ratio],
    'SVI': [man_svi, bk_svi, queens_svi, bx_svi, si_svi],
    'Housing and Transportation SVI': [man_ht_svi, bk_ht_svi, queens_ht_svi, bx_ht_svi, si_ht_svi]
}
data_df = pd.DataFrame(data)
data_df

,Borough,Population,Area,Train Stations,Bus Stops,Stops Ratio,Bus Stops Ratio,SVI,Housing and Transportation SVI
0,Manhattan,1645867,22.65796344,153,720,0.09296012,0.43745941,0.85250000,0.98360000
1,Brooklyn,2679620,69.37653864,169,871,0.06306864,0.32504609,0.96720000,0.80330000
2,Queens,2360826,108.72402512,83,968,0.03515719,0.41002598,0.88520000,0.72130000
3,Bronx,1443229,42.17611220,70,563,0.04850235,0.39009748,1.00000000,0.95080000
4,Staten Island,492925,57.52251071,21,208,0.04260283,0.42197089,0.70490000,0.68850000
